# Data cleaning and modelling

In [222]:
#import all reuired packages
import pandas as pd
from functools import reduce

## Task 
Analyse content categories that highlights the top 5 categories with the largest aggregate popularity
Tables needed:
- Reactions
- Content
- ReactionTypes

In [223]:
#load required datasets
content_df = pd.read_csv('./data/Content.csv')
reaction_df = pd.read_csv('./data/Reactions.csv')
types_df = pd.read_csv('./data/ReactionTypes.csv')

## Data quality assessment
- data types (mismatched, mixed data types)
- missing data
- rename columns for ease of access

In [224]:
content_df.columns

Index(['Unnamed: 0', 'Content ID', 'User ID', 'Type', 'Category', 'URL'], dtype='object')

In [225]:
content_df.drop(columns=['Unnamed: 0'], inplace=True)

In [226]:
content_df.head()

,Content ID,User ID,Type,Category,URL
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,photo,Studying,https://socialbuzz.cdn.com/content/storage/975...
1,9f737e0a-3cdd-4d29-9d24-753f4e3be810,beb1f34e-7870-46d6-9fc7-2e12eb83ce43,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/9f7...
2,230c4e4d-70c3-461d-b42c-ec09396efb3f,a5c65404-5894-4b87-82f2-d787cbee86b4,photo,healthy eating,https://socialbuzz.cdn.com/content/storage/230...
3,356fff80-da4d-4785-9f43-bc1261031dc6,9fb4ce88-fac1-406c-8544-1a899cee7aaf,photo,technology,https://socialbuzz.cdn.com/content/storage/356...
4,01ab84dd-6364-4236-abbb-3f237db77180,e206e31b-5f85-4964-b6ea-d7ee5324def1,video,food,https://socialbuzz.cdn.com/content/storage/01a...


In [227]:
#filter out important columns
content_df = content_df[['Content ID','User ID','Category', 'Type']]

In [228]:
content_df = content_df.rename(columns={'Type': 'Content Type'})

In [229]:
#check for missing values
content_df.isna().sum()

Content ID      0
User ID         0
Category        0
Content Type    0
dtype: int64

In [230]:
reaction_df.columns

Index(['Unnamed: 0', 'Content ID', 'User ID', 'Type', 'Datetime'], dtype='object')

In [231]:
reaction_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [232]:
reaction_df.isna().sum()

Content ID       0
User ID       3019
Type           980
Datetime         0
dtype: int64

In [233]:
#exclude missing values for userID
reaction_df = reaction_df[reaction_df['User ID'].notna()]

In [234]:
reaction_df.isna().sum()

Content ID    0
User ID       0
Type          0
Datetime      0
dtype: int64

In [235]:
reaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22534 entries, 1 to 25552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Content ID  22534 non-null  object
 1   User ID     22534 non-null  object
 2   Type        22534 non-null  object
 3   Datetime    22534 non-null  object
dtypes: object(4)
memory usage: 880.2+ KB


In [236]:
reaction_df['Datetime'] = reaction_df['Datetime'].astype('datetime64[ns]')
reaction_df['Type'] = reaction_df['Type'].astype('category')

In [237]:
reaction_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22534 entries, 1 to 25552
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Content ID  22534 non-null  object        
 1   User ID     22534 non-null  object        
 2   Type        22534 non-null  category      
 3   Datetime    22534 non-null  datetime64[ns]
dtypes: category(1), datetime64[ns](1), object(2)
memory usage: 726.9+ KB


In [238]:
reaction_df.head()

,Content ID,User ID,Type,Datetime
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,5d454588-283d-459d-915d-c48a2cb4c27f,disgust,2020-11-07 09:43:50
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,92b87fa5-f271-43e0-af66-84fac21052e6,dislike,2021-06-17 12:22:51
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared,2021-04-18 05:13:58
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,34e8add9-0206-47fd-a501-037b994650a2,disgust,2021-01-06 19:13:01
5,97522e57-d9ab-4bd6-97bf-c24d952602d2,9b6d35f9-5e15-4cd0-a8d7-b1f3340e02c4,interested,2020-08-23 12:25:58


In [239]:
#exclude non important columns
reaction_df = reaction_df[['Content ID', 'User ID', 'Type']]

In [240]:
types_df.columns

Index(['Unnamed: 0', 'Type', 'Sentiment', 'Score'], dtype='object')

In [241]:
types_df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)

In [242]:
types_df.isna().sum()

Type         0
Sentiment    0
Score        0
dtype: int64

In [243]:
types_df.shape

(16, 3)

In [244]:
types_df.head()

,Type,Sentiment,Score
0,heart,positive,60
1,want,positive,70
2,disgust,negative,0
3,hate,negative,5
4,interested,positive,30


### Preparing to merge tables

In [245]:
#shows all reactions that have been made by users
reaction_df.columns

Index(['Content ID', 'User ID', 'Type'], dtype='object')

In [246]:
#defines reactions
types_df.columns

Index(['Type', 'Sentiment', 'Score'], dtype='object')

In [247]:
#shows all content that have been saved with category
content_df.columns

Index(['Content ID', 'User ID', 'Category', 'Content Type'], dtype='object')

In [248]:
#which content has been reacted to
#we want only the content that has reactions
content_reaction = content_df.merge(reaction_df, on='Content ID',how='left')

In [249]:
content_reaction.shape

(22572, 6)

In [250]:
content_reaction.head()

,Content ID,User ID_x,Category,Content Type,User ID_y,Type
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,5d454588-283d-459d-915d-c48a2cb4c27f,disgust
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,92b87fa5-f271-43e0-af66-84fac21052e6,dislike
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,163daa38-8b77-48c9-9af6-37a6c1447ac2,scared
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,34e8add9-0206-47fd-a501-037b994650a2,disgust
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,9b6d35f9-5e15-4cd0-a8d7-b1f3340e02c4,interested


In [251]:
content_reaction.drop(columns=['User ID_y'], inplace=True)

In [252]:

final_dataset = content_reaction.merge(types_df, on='Type', how='left')

In [253]:
final_dataset.head()

,Content ID,User ID_x,Category,Content Type,Type,Sentiment,Score
0,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,disgust,negative,0.0
1,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,dislike,negative,10.0
2,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,scared,negative,15.0
3,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,disgust,negative,0.0
4,97522e57-d9ab-4bd6-97bf-c24d952602d2,8d3cd87d-8a31-4935-9a4f-b319bfe05f31,Studying,photo,interested,positive,30.0


In [254]:
final_dataset = final_dataset.rename(columns={'User ID_x': 'User_ID'})
#final_dataset = final_dataset.astype({'Score':'int64'})

In [255]:
final_dataset.isna().sum()

Content ID       0
User_ID          0
Category         0
Content Type     0
Type            38
Sentiment       38
Score           38
dtype: int64

In [256]:
final_dataset = final_dataset[final_dataset['Type'].notna()]

In [257]:
final_dataset = final_dataset.astype({'Score':'int64'})

In [258]:
final_dataset.shape

(22534, 7)

In [259]:
#categories with largest aggregate by popularity
final_dataset.groupby('Category')['Score'].sum().sort_values(ascending=False)[:5].to_csv('./data/top5.csv')

In [260]:
final_dataset.to_csv('./data/merged_data.csv', index=False)

In [261]:
#merge all data on User ID
#data_merged = reduce(lambda left,right: pd.merge(left,right,on='User ID'), data)

In [262]:
#check = content_df[content_df.isnull().any(axis=1)]